In [1]:
from dotenv import load_dotenv
import os

# Load environment variables from the .env file
load_dotenv()

# Access variables
TWILIO_ACCOUNT_SID = os.getenv('TWILIO_ACCOUNT_SID')
TWILIO_AUTH_TOKEN = os.getenv('TWILIO_AUTH_TOKEN')
TWILIO_PHONE_NUMBER = os.getenv('TWILIO_PHONE_NUMBER')
EMERGENCY_NUMBER = os.getenv('EMERGENCY_NUMBER')

In [3]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
import time
from twilio.rest import Client
from datetime import datetime
import os
from keras import backend as K

# Custom metric functions
def recall_m(y_true, y_pred):
  true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
  possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
  recall = true_positives / (possible_positives + K.epsilon())
  return recall

def precision_m(y_true, y_pred):
  true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
  predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
  precision = true_positives / (predicted_positives + K.epsilon())
  return precision

def f1_m(y_true, y_pred):
  precision = precision_m(y_true, y_pred)
  recall = recall_m(y_true, y_pred)
  return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [4]:

# Twilio credentials

def verify_twilio_credentials():
  """Verify Twilio credentials are working"""
  try:
      client = Client(TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN)
      # Try to access account info to verify credentials
      account = client.api.accounts(TWILIO_ACCOUNT_SID).fetch()
      print("Twilio credentials verified successfully")
      return True
  except Exception as e:
      print(f"Twilio credential verification failed: {str(e)}")
      return False

def send_emergency_sms(source_type="webcam"):
  """Send an emergency SMS using Twilio with enhanced error handling"""
  try:
      # First verify credentials
      if not verify_twilio_credentials():
          print("Skipping SMS send due to invalid credentials")
          return False

      # Initialize Twilio client
      client = Client(TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN)
      
      # Prepare message
      message_body = f"⚠️ ALERT: Fight detected in {source_type} footage! Please check immediately."
      
      # Send message with detailed error handling
      try:
          message = client.messages.create(
              body=message_body,
              from_=TWILIO_PHONE_NUMBER,
              to=EMERGENCY_NUMBER
          )
          print(f"Emergency SMS sent successfully - SID: {message.sid}")
          return True
          
      except Exception as msg_error:
          if 'unauthorized' in str(msg_error).lower():
              print("Error: Unauthorized. Please check your Twilio credentials.")
          elif 'not a valid phone number' in str(msg_error).lower():
              print(f"Error: Invalid phone number format: {EMERGENCY_NUMBER}")
          elif 'not a valid twilio phone number' in str(msg_error).lower():
              print(f"Error: Invalid Twilio phone number: {TWILIO_PHONE_NUMBER}")
          else:
              print(f"SMS sending failed: {str(msg_error)}")
          return False
          
  except Exception as e:
      print(f"Failed to initialize Twilio client: {str(e)}")
      return False

def make_emergency_call():
  """Make an emergency call using Twilio with enhanced error handling"""
  try:
      # First verify credentials
      if not verify_twilio_credentials():
          print("Skipping call due to invalid credentials")
          return False

      # Initialize Twilio client
      client = Client(TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN)
      
      try:
          call = client.calls.create(
              url='http://demo.twilio.com/docs/voice.xml',
              to=EMERGENCY_NUMBER,
              from_=TWILIO_PHONE_NUMBER
          )
          print(f"Emergency call initiated successfully - SID: {call.sid}")
          return True
          
      except Exception as call_error:
          if 'unauthorized' in str(call_error).lower():
              print("Error: Unauthorized. Please check your Twilio credentials.")
          elif 'not a valid phone number' in str(call_error).lower():
              print(f"Error: Invalid phone number format: {EMERGENCY_NUMBER}")
          elif 'not a valid twilio phone number' in str(call_error).lower():
              print(f"Error: Invalid Twilio phone number: {TWILIO_PHONE_NUMBER}")
          else:
              print(f"Call initiation failed: {str(call_error)}")
          return False
          
  except Exception as e:
      print(f"Failed to initialize Twilio client: {str(e)}")
      return False

In [5]:


def process_video_feed(frame, model, frames_buffer, segment_frames=42):
  """Process video frame and make prediction with enhanced debugging"""
  try:
      # Resize frame to match model's expected input
      resized_frame = cv2.resize(frame, (128, 128))
      frames_buffer.append(resized_frame)
      
      if len(frames_buffer) > segment_frames:
          frames_buffer.pop(0)
      
      if len(frames_buffer) == segment_frames:
          # Prepare video segment
          video_segment = np.array(frames_buffer)
          video_segment = video_segment.astype('float32') / 255.0
          video_segment = np.expand_dims(video_segment, axis=0)
          
          # Get prediction
          prediction = model.predict(video_segment, verbose=0)
          
          # Get probabilities for both classes
          no_fight_prob = prediction[0][0]
          fight_prob = prediction[0][1]
          
          # Use fight probability for confidence
          predicted_class = 1 if fight_prob > 0.4 else 0  # Adjusted threshold
          confidence = fight_prob  # Use fight probability directly
          
          # Debug information
          print("\nPrediction Analysis:")
          print(f"No Fight Probability: {no_fight_prob:.2%}")
          print(f"Fight Probability: {fight_prob:.2%}")
          print(f"Predicted class: {'Fight' if predicted_class == 1 else 'No Fight'}")
          print(f"Confidence: {confidence:.2%}")
          
          return predicted_class, confidence
      else:
          print(f"\rCollecting frames: {len(frames_buffer)}/{segment_frames}", end="")
      
      return None, 0.0
      
  except Exception as e:
      print(f"Error in process_video_feed: {str(e)}")
      return None, 0.0

In [6]:


def monitor_video_file(video_path, model_path, confidence_threshold=0.75, cooldown_period=60):
  """Monitor video file for fight detection (headless version)"""
  if not os.path.exists(video_path):
      print(f"Error: Video file not found at {video_path}")
      return
  
  print(f"Starting video file surveillance: {video_path}")
  print(f"Emergency contact number: {EMERGENCY_NUMBER}")
  
  # Create output directory for detected frames
  output_dir = 'detected_fights'
  os.makedirs(output_dir, exist_ok=True)
  
  try:
      model = keras.models.load_model(model_path, 
                                  custom_objects={'recall_m': recall_m, 
                                                'precision_m': precision_m, 
                                                'f1_m': f1_m})
  except Exception as e:
      print(f"Error loading model: {str(e)}")
      return
  
  cap = cv2.VideoCapture(video_path)
  if not cap.isOpened():
      print("Error: Could not open video file")
      return

  frames_buffer = []
  last_alert_time = 0
  frame_count = 0
  
  print("Processing video... Press Ctrl+C to stop")
  
  try:
      while True:
          ret, frame = cap.read()
          if not ret:
              break
          
          frame_count += 1
          if frame_count % 100 == 0:  # Progress update every 100 frames
              print(f"Processed {frame_count} frames")
          
          predicted_class, confidence = process_video_feed(frame, model, frames_buffer)
          
          if predicted_class is not None:
              current_time = time.time()
              if predicted_class == 1 and confidence >= confidence_threshold:
                  if current_time - last_alert_time >= cooldown_period:
                      print(f"\nFight detected in video!")
                      print(f"Frame: {frame_count}")
                      print(f"Confidence: {confidence:.2%}")
                      print(f"Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
                      
                      # Send alerts
                      alert_sent = False
                      if make_emergency_call():
                          if send_emergency_sms("video file"):
                              alert_sent = True
                              last_alert_time = current_time
                      
                      # Save detected frame
                      timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
                      output_path = os.path.join(output_dir, 
                                             f'fight_frame{frame_count}_{timestamp}.jpg')
                      cv2.imwrite(output_path, frame)
                      print(f"Saved detection frame to: {output_path}")
  
  except KeyboardInterrupt:
      print("\nProcessing interrupted by user")
  except Exception as e:
      print(f"\nError during video processing: {str(e)}")
  finally:
      cap.release()
      print(f"\nProcessing complete. Total frames processed: {frame_count}")
      print(f"Detected frames saved in: {output_dir}")



In [7]:
def monitor_webcam(model_path, confidence_threshold=0.4):  # Lowered default threshold
  """Monitor webcam feed for fight detection with enhanced visualization"""
  print("Initializing webcam surveillance...")
  
  try:
      model = keras.models.load_model(model_path, 
                                  custom_objects={'recall_m': recall_m, 
                                                'precision_m': precision_m, 
                                                'f1_m': f1_m})
      print("Model loaded successfully")
      print("Model summary:")
      model.summary()
  except Exception as e:
      print(f"Error loading model: {str(e)}")
      return
  
  # Initialize webcam
  cap = cv2.VideoCapture(0)
  if not cap.isOpened():
      print("Error: Could not access webcam")
      return

  # Set webcam resolution
  cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
  cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

  frames_buffer = []
  last_alert_time = 0
  last_process_time = 0
  process_interval = 1  # Process every second
  fps_counter = 0
  fps_timer = time.time()
  fps = 0
  status = "Initializing..."
  processing_time = 0
  confidence = 0.0
  fight_prob = 0.0
  no_fight_prob = 0.0
  
  print("\nMonitoring started... Press 'q' to quit")
  print(f"Confidence threshold: {confidence_threshold:.2%}")
  print("Collecting initial frames...")
  
  try:
      while True:
          ret, frame = cap.read()
          if not ret:
              print("Error reading from webcam")
              break
          
          current_time = time.time()
          
          # Calculate FPS
          fps_counter += 1
          if current_time - fps_timer >= 1.0:
              fps = fps_counter
              fps_counter = 0
              fps_timer = current_time
          
          # Create display frame with info
          display_frame = frame.copy()
          
          # Process frame every second
          if current_time - last_process_time >= process_interval:
              status = "Processing..."
              process_start = time.time()
              
              pred_class, pred_confidence = process_video_feed(frame, model, frames_buffer)
              
              if pred_class is not None:
                  confidence = pred_confidence
                  
                  if pred_class == 1 and confidence >= confidence_threshold:
                      status = "⚠️ FIGHT DETECTED!"
                      # Add red border and warning text
                      display_frame = cv2.rectangle(display_frame, (0, 0), 
                                                  (display_frame.shape[1], display_frame.shape[0]), 
                                                  (0, 0, 255), 10)
                      
                      # Add large warning text
                      cv2.putText(display_frame, "FIGHT DETECTED!", 
                                (display_frame.shape[1]//4, display_frame.shape[0]//2),
                                cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 3)
                      
                      if current_time - last_alert_time >= 30:
                          print("\n⚠️ FIGHT DETECTED!")
                          print(f"Fight Probability: {confidence:.2%}")
                          print(f"Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
                          
                          if send_emergency_sms("webcam"):
                              print("Alert SMS sent successfully")
                              last_alert_time = current_time
                  else:
                      status = "No Fight Detected"
              
              processing_time = time.time() - process_start
              last_process_time = current_time
          
          # Add status overlay with more information
          cv2.putText(display_frame, f"Status: {status}", (10, 30), 
                     cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
          cv2.putText(display_frame, f"FPS: {fps}", (10, 60), 
                     cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
          cv2.putText(display_frame, f"Fight Probability: {confidence:.2%}", (10, 90), 
                     cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
          cv2.putText(display_frame, f"Threshold: {confidence_threshold:.2%}", (10, 120), 
                     cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
          
          # Create a resizable window
          cv2.namedWindow('Fight Detection Monitor', cv2.WINDOW_NORMAL)
          cv2.resizeWindow('Fight Detection Monitor', 800, 600)
          cv2.imshow('Fight Detection Monitor', display_frame)
          
          # Check for 'q' key to quit
          if cv2.waitKey(1) & 0xFF == ord('q'):
              print("\nStopping surveillance...")
              break
              
  except KeyboardInterrupt:
      print("\nSurveillance interrupted by user")
  except Exception as e:
      print(f"\nError during surveillance: {str(e)}")
      raise
  finally:
      cap.release()
      cv2.destroyAllWindows()

In [ ]:


def main():
  model_path = 'vivit_model'
  
  while True:
      print("\nFight Detection System")
      print("1. Monitor Webcam")
      print("2. Process Video File")
      print("3. Test SMS Alert")
      print("4. Exit")
      
      choice = input("Enter your choice (1-4): ")
      
      if choice == '1':
          threshold = float(input("Enter confidence threshold (0.1-0.9, default 0.4): ") or 0.4)
          monitor_webcam(model_path, confidence_threshold=threshold)
      elif choice == '2':
          video_path = input("Enter the path to video file: ")
          monitor_video_file(video_path, model_path)
      elif choice == '3':
          print("Testing SMS alert system...")
          if send_emergency_sms("test"):
              print("SMS test successful")
          else:
              print("SMS test failed")
      elif choice == '4':
          print("Exiting...")
          break
      else:
          print("Invalid choice. Please try again.")

if __name__ == "__main__":
  main()


Fight Detection System
1. Monitor Webcam
2. Process Video File
3. Test SMS Alert
4. Exit
Initializing webcam surveillance...
Model loaded successfully
Model summary:
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 42, 128, 128, 3)]    0         []                            
                                                                                                  
 tubelet_embedding (Tubelet  (None, 1280, 64)             98368     ['input_1[0][0]']             
 Embedding)                                                                                       
                                                                                                  
 positional_encoder (Positi  (None, 1280, 64)             81920     ['tubelet_embedding[0][0]']   
 onalEncoder)             

2024-11-23 23:50:37,046 - INFO - -- BEGIN Twilio API Request --
2024-11-23 23:50:37,047 - INFO - GET Request: https://api.twilio.com/2010-04-01/Accounts/AC1c7a6f255c2df8ec487b6c2fa27087e5.json
2024-11-23 23:50:37,048 - INFO - Headers:
2024-11-23 23:50:37,049 - INFO - User-Agent : twilio-python/9.3.7 (Windows AMD64) Python/3.10.11
2024-11-23 23:50:37,049 - INFO - X-Twilio-Client : python-9.3.7
2024-11-23 23:50:37,051 - INFO - Accept-Charset : utf-8
2024-11-23 23:50:37,052 - INFO - Accept : application/json
2024-11-23 23:50:37,053 - INFO - -- END Twilio API Request --



Prediction Analysis:
No Fight Probability: 59.85%
Fight Probability: 40.15%
Predicted class: Fight
Confidence: 40.15%

⚠️ FIGHT DETECTED!
Fight Probability: 40.15%
Time: 2024-11-23 23:50:37


2024-11-23 23:50:37,438 - INFO - Response Status Code: 200
2024-11-23 23:50:37,440 - INFO - Response Headers: {'Content-Type': 'application/json; charset=utf-8', 'Content-Length': '2261', 'Connection': 'keep-alive', 'Date': 'Sat, 23 Nov 2024 18:20:37 GMT', 'X-API-Domain': 'api.twilio.com', 'Last-Modified': 'Tue, 19 Nov 2024 22:08:20 +0000', 'Etag': '57566558290a7657b31c9186b75b2679', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Headers': 'Accept, Authorization, Content-Type, If-Match, If-Modified-Since, If-None-Match, If-Unmodified-Since, Idempotency-Key', 'Access-Control-Allow-Credentials': 'true', 'Access-Control-Allow-Methods': 'GET, POST, DELETE, OPTIONS', 'Access-Control-Expose-Headers': 'ETag', 'Twilio-Request-Id': 'RQe7484ea70c93038407b8c8ca547f8e97', 'Twilio-Request-Duration': '0.014', 'Twilio-Concurrent-Requests': '1', 'X-Shenanigans': 'none', 'X-Powered-By': 'AT-5000', 'X-Home-Region': 'us1', 'X-Cache': 'Miss from cloudfront', 'Via': '1.1 b58fb22cae083768666eb24f

Twilio credentials verified successfully


...


Prediction Analysis:
No Fight Probability: 56.26%
Fight Probability: 43.74%
Predicted class: Fight
Confidence: 43.74%

⚠️ FIGHT DETECTED!
Fight Probability: 43.74%
Time: 2024-11-23 23:51:08


2024-11-23 23:51:08,323 - INFO - Response Status Code: 200
2024-11-23 23:51:08,325 - INFO - Response Headers: {'Content-Type': 'application/json; charset=utf-8', 'Content-Length': '2261', 'Connection': 'keep-alive', 'Date': 'Sat, 23 Nov 2024 18:21:08 GMT', 'X-API-Domain': 'api.twilio.com', 'Last-Modified': 'Tue, 19 Nov 2024 22:08:20 +0000', 'Etag': '57566558290a7657b31c9186b75b2679', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Headers': 'Accept, Authorization, Content-Type, If-Match, If-Modified-Since, If-None-Match, If-Unmodified-Since, Idempotency-Key', 'Access-Control-Allow-Credentials': 'true', 'Access-Control-Allow-Methods': 'GET, POST, DELETE, OPTIONS', 'Access-Control-Expose-Headers': 'ETag', 'Twilio-Request-Id': 'RQ2e3aa4f29c44d2463f51d0a29d5e1b64', 'Twilio-Request-Duration': '0.012', 'Twilio-Concurrent-Requests': '1', 'X-Shenanigans': 'none', 'X-Powered-By': 'AT-5000', 'X-Home-Region': 'us1', 'X-Cache': 'Miss from cloudfront', 'Via': '1.1 38e78452b9b62c839c58248b

Twilio credentials verified successfully


2024-11-23 23:51:08,737 - INFO - Response Status Code: 201
2024-11-23 23:51:08,737 - INFO - Response Headers: {'Content-Type': 'application/json;charset=utf-8', 'Content-Length': '882', 'Connection': 'keep-alive', 'Date': 'Sat, 23 Nov 2024 18:21:09 GMT', 'X-Powered-By': 'AT-5000', 'Twilio-Concurrent-Requests': '1', 'Twilio-Request-Id': 'RQ783245dc91802955e8d7da3366f5d1a2', 'Twilio-Request-Duration': '0.128', 'X-Home-Region': 'us1', 'X-Api-Domain': 'api.twilio.com', 'Strict-Transport-Security': 'max-age=31536000', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Headers': 'Accept, Authorization, Content-Type, If-Match, If-Modified-Since, If-None-Match, If-Unmodified-Since, Idempotency-Key, X-Pre-Auth-Context, X-Target-Region', 'Access-Control-Allow-Methods': 'GET, POST, PATCH, PUT, DELETE, OPTIONS', 'Access-Control-Expose-Headers': 'ETag, Twilio-Request-Id', 'Access-Control-Allow-Credentials': 'true', 'X-Shenanigans': 'none', 'X-Cache': 'Miss from cloudfront', 'Via': '1.1 4ba2a

Emergency SMS sent successfully - SID: SM783245dc91802955e8d7da3366f5d1a2
Alert SMS sent successfully

Prediction Analysis:
No Fight Probability: 56.65%
Fight Probability: 43.35%
Predicted class: Fight
Confidence: 43.35%

Prediction Analysis:
No Fight Probability: 57.10%
Fight Probability: 42.90%
Predicted class: Fight
Confidence: 42.90%

Prediction Analysis:
No Fight Probability: 57.49%
Fight Probability: 42.51%
Predicted class: Fight
Confidence: 42.51%

Prediction Analysis:
No Fight Probability: 57.93%
Fight Probability: 42.07%
Predicted class: Fight
Confidence: 42.07%

Prediction Analysis:
No Fight Probability: 58.30%
Fight Probability: 41.70%
Predicted class: Fight
Confidence: 41.70%

Prediction Analysis:
No Fight Probability: 58.69%
Fight Probability: 41.31%
Predicted class: Fight
Confidence: 41.31%

Prediction Analysis:
No Fight Probability: 59.10%
Fight Probability: 40.90%
Predicted class: Fight
Confidence: 40.90%

Prediction Analysis:
No Fight Probability: 60.03%
Fight Probabil

In [ ]:
%pip install playsound

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for playsound: filename=playsound-1.3.0-py3-none-any.whl size=7044 sha256=9ad26d61c86174b0ade1fc0657c4020bcbf9d74cc8d71bca4f45020589936ff3
  Stored in directory: c:\users\gupta\appdata\local\pip\cache\wheels\90\89\ed\2d643f4226fc8c7c9156fc28abd8051e2d2c0de37ae51ac45c
Successfully built playsound
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
import time
from datetime import datetime
import os
from twilio.rest import Client
import logging

# Configure logging
logging.basicConfig(
  level=logging.INFO,
  format='%(asctime)s - %(levelname)s - %(message)s',
  handlers=[
      logging.FileHandler('fight_detection.log'),
      logging.StreamHandler()
  ]
)

# Twilio configuration

# Metrics functions for model loading
def recall_m(y_true, y_pred):
  true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
  possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
  recall = true_positives / (possible_positives + K.epsilon())
  return recall

def precision_m(y_true, y_pred):
  true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
  predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
  precision = true_positives / (predicted_positives + K.epsilon())
  return precision

def f1_m(y_true, y_pred):
  precision = precision_m(y_true, y_pred)
  recall = recall_m(y_true, y_pred)
  return 2*((precision*recall)/(precision+recall+K.epsilon()))

def send_emergency_alert(detection_type="webcam", location="Unknown"):
  """Send both SMS and voice call for emergency alert"""
  try:
      client = Client(TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN)
      
      # Send SMS
      message_text = (f"⚠️ EMERGENCY ALERT! Fight detected via {detection_type} "
                     f"at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')} "
                     f"Location: {location}")
      
      message = client.messages.create(
          body=message_text,
          from_=TWILIO_PHONE_NUMBER,
          to=EMERGENCY_NUMBER
      )
      logging.info(f"SMS sent successfully! SID: {message.sid}")
      
      # Make voice call
      call = client.calls.create(
          twiml=f'<Response><Say>Emergency! Fight detected via {detection_type}. '
                f'Please check the location immediately.</Say></Response>',
          from_=TWILIO_PHONE_NUMBER,
          to=EMERGENCY_NUMBER
      )
      logging.info(f"Call initiated! SID: {call.sid}")
      
      return True
      
  except Exception as e:
      logging.error(f"Error sending emergency alert: {str(e)}")
      return False

def verify_twilio_credentials():
  """Verify Twilio credentials are working"""
  try:
      client = Client(TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN)
      client.api.accounts(TWILIO_ACCOUNT_SID).fetch()
      return True
  except Exception as e:
      logging.error(f"Twilio verification failed: {str(e)}")
      return False

def process_video_feed(frame, model, frames_buffer, segment_frames=42):
  """Process video frame and make prediction with enhanced debugging"""
  try:
      resized_frame = cv2.resize(frame, (128, 128))
      frames_buffer.append(resized_frame)
      
      if len(frames_buffer) > segment_frames:
          frames_buffer.pop(0)
      
      if len(frames_buffer) == segment_frames:
          video_segment = np.array(frames_buffer)
          video_segment = video_segment.astype('float32') / 255.0
          video_segment = np.expand_dims(video_segment, axis=0)
          
          prediction = model.predict(video_segment, verbose=0)
          
          # Get probabilities for both classes
          no_fight_prob = prediction[0][0]
          fight_prob = prediction[0][1]
          
          # Use fight probability for confidence
          predicted_class = 1 if fight_prob > 0.4 else 0
          confidence = fight_prob
          
          logging.debug(f"\nPrediction Analysis:")
          logging.debug(f"No Fight Probability: {no_fight_prob:.2%}")
          logging.debug(f"Fight Probability: {fight_prob:.2%}")
          logging.debug(f"Predicted class: {'Fight' if predicted_class == 1 else 'No Fight'}")
          logging.debug(f"Confidence: {confidence:.2%}")
          
          return predicted_class, confidence
      else:
          print(f"\rCollecting frames: {len(frames_buffer)}/{segment_frames}", end="")
      
      return None, 0.0
      
  except Exception as e:
      logging.error(f"Error in process_video_feed: {str(e)}")
      return None, 0.0

def monitor_webcam(model_path, confidence_threshold=0.4):
  """Monitor webcam feed for fight detection with enhanced alerts"""
  logging.info("Initializing webcam surveillance...")
  
  try:
      model = keras.models.load_model(model_path, 
                                  custom_objects={'recall_m': recall_m, 
                                                'precision_m': precision_m, 
                                                'f1_m': f1_m})
      logging.info("Model loaded successfully")
  except Exception as e:
      logging.error(f"Error loading model: {str(e)}")
      return
  
  cap = cv2.VideoCapture(0)
  if not cap.isOpened():
      logging.error("Error: Could not access webcam")
      return

  cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
  cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

  frames_buffer = []
  last_alert_time = 0
  last_process_time = 0
  process_interval = 1
  fps_counter = 0
  fps_timer = time.time()
  fps = 0
  status = "Monitoring"
  processing_time = 0
  confidence = 0.0
  alert_cooldown = 60
  
  logging.info("\nMonitoring started... Press 'q' to quit")
  logging.info(f"Confidence threshold: {confidence_threshold:.2%}")
  
  try:
      while True:
          ret, frame = cap.read()
          if not ret:
              logging.error("Error reading from webcam")
              break
          
          current_time = time.time()
          
          fps_counter += 1
          if current_time - fps_timer >= 1.0:
              fps = fps_counter
              fps_counter = 0
              fps_timer = current_time
          
          display_frame = frame.copy()
          
          if current_time - last_process_time >= process_interval:
              status = "Processing..."
              process_start = time.time()
              
              pred_class, pred_confidence = process_video_feed(frame, model, frames_buffer)
              
              if pred_class is not None:
                  confidence = pred_confidence
                  
                  if pred_class == 1 and confidence >= confidence_threshold:
                      status = "⚠️ FIGHT DETECTED!"
                      display_frame = cv2.rectangle(display_frame, (0, 0), 
                                                  (display_frame.shape[1], display_frame.shape[0]), 
                                                  (0, 0, 255), 10)
                      
                      cv2.putText(display_frame, "FIGHT DETECTED!", 
                                (display_frame.shape[1]//4, display_frame.shape[0]//2),
                                cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 3)
                      
                      if current_time - last_alert_time >= alert_cooldown:
                          logging.warning("\n⚠️ FIGHT DETECTED!")
                          logging.info(f"Confidence: {confidence:.2%}")
                          logging.info(f"Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
                          
                          if send_emergency_alert("webcam"):
                              logging.info("Emergency alerts (SMS and Call) sent successfully")
                              last_alert_time = current_time
                  else:
                      status = "No Fight Detected"
              
              processing_time = time.time() - process_start
              last_process_time = current_time
          
          # Add status overlay
          cv2.putText(display_frame, f"Status: {status}", (10, 30), 
                     cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
          cv2.putText(display_frame, f"FPS: {fps}", (10, 60), 
                     cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
          cv2.putText(display_frame, f"Confidence: {confidence:.2%}", (10, 90), 
                     cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
          
          cv2.namedWindow('Fight Detection Monitor', cv2.WINDOW_NORMAL)
          cv2.resizeWindow('Fight Detection Monitor', 800, 600)
          cv2.imshow('Fight Detection Monitor', display_frame)
          
          if cv2.waitKey(1) & 0xFF == ord('q'):
              logging.info("\nStopping surveillance...")
              break
              
  except KeyboardInterrupt:
      logging.info("\nSurveillance interrupted by user")
  except Exception as e:
      logging.error(f"\nError during surveillance: {str(e)}")
      raise
  finally:
      cap.release()
      cv2.destroyAllWindows()

def monitor_video_file(video_path, model_path, confidence_threshold=0.4):
  """Process a video file for fight detection"""
  logging.info(f"Processing video file: {video_path}")
  
  try:
      model = keras.models.load_model(model_path, 
                                  custom_objects={'recall_m': recall_m, 
                                                'precision_m': precision_m, 
                                                'f1_m': f1_m})
      logging.info("Model loaded successfully")
  except Exception as e:
      logging.error(f"Error loading model: {str(e)}")
      return
  
  cap = cv2.VideoCapture(video_path)
  if not cap.isOpened():
      logging.error("Error: Could not open video file")
      return
  
  frames_buffer = []
  last_alert_time = 0
  
  try:
      while cap.isOpened():
          ret, frame = cap.read()
          if not ret:
              break
          
          pred_class, confidence = process_video_feed(frame, model, frames_buffer)
          
          if pred_class is not None and pred_class == 1 and confidence >= confidence_threshold:
              current_time = time.time()
              if current_time - last_alert_time >= 60:
                  logging.warning(f"\n⚠️ Fight detected at {cap.get(cv2.CAP_PROP_POS_MSEC)/1000:.2f} seconds")
                  logging.info(f"Confidence: {confidence:.2%}")
                  send_emergency_alert("video")
                  last_alert_time = current_time
          
          cv2.imshow('Video Processing', frame)
          if cv2.waitKey(1) & 0xFF == ord('q'):
              break
              
  finally:
      cap.release()
      cv2.destroyAllWindows()

def main():
  while True:
      print("\nFight Detection System")
      print("1. Monitor Webcam")
      print("2. Process Video File")
      print("3. Test Emergency Alerts (SMS + Call)")
      print("4. Exit")
      
      choice = input("Enter your choice (1-4): ")
      
      if choice == '1':
          if not verify_twilio_credentials():
              logging.warning("Warning: Twilio setup is not correct. Alerts will not be sent.")
              proceed = input("Do you want to continue anyway? (y/n): ")
              if proceed.lower() != 'y':
                  continue
          
          threshold = float(input("Enter confidence threshold (0.1-0.9, default 0.4): ") or 0.4)
          monitor_webcam('vivit_model', confidence_threshold=threshold)
      
      elif choice == '2':
          video_path = input("Enter the path to video file: ")
          monitor_video_file(video_path, 'vivit_model')
      
      elif choice == '3':
          logging.info("Testing emergency alerts...")
          if send_emergency_alert("test"):
              logging.info("Emergency alerts test successful")
          else:
              logging.error("Emergency alerts test failed")
      
      elif choice == '4':
          logging.info("Exiting...")
          break
      
      else:
          logging.warning("Invalid choice. Please try again.")

if __name__ == "__main__":
  main()


Fight Detection System
1. Monitor Webcam
2. Process Video File
3. Test Emergency Alerts (SMS + Call)
4. Exit


2024-11-23 23:48:08,408 - INFO - -- BEGIN Twilio API Request --
2024-11-23 23:48:08,409 - INFO - GET Request: https://api.twilio.com/2010-04-01/Accounts/AC1c7a6f255c2df8ec487b6c2fa27087e5.json
2024-11-23 23:48:08,409 - INFO - Headers:
2024-11-23 23:48:08,410 - INFO - User-Agent : twilio-python/9.3.7 (Windows AMD64) Python/3.10.11
2024-11-23 23:48:08,411 - INFO - X-Twilio-Client : python-9.3.7
2024-11-23 23:48:08,412 - INFO - Accept-Charset : utf-8
2024-11-23 23:48:08,412 - INFO - Accept : application/json
2024-11-23 23:48:08,413 - INFO - -- END Twilio API Request --
2024-11-23 23:48:08,727 - INFO - Response Status Code: 200
2024-11-23 23:48:08,728 - INFO - Response Headers: {'Content-Type': 'application/json; charset=utf-8', 'Content-Length': '2261', 'Connection': 'keep-alive', 'Date': 'Sat, 23 Nov 2024 18:18:09 GMT', 'X-API-Domain': 'api.twilio.com', 'Last-Modified': 'Tue, 19 Nov 2024 22:08:20 +0000', 'Etag': '57566558290a7657b31c9186b75b2679', 'Access-Control-Allow-Origin': '*', 'Acc

2024-11-23 23:48:35,543 - INFO - 
Stopping surveillance...



Fight Detection System
1. Monitor Webcam
2. Process Video File
3. Test Emergency Alerts (SMS + Call)
4. Exit


2024-11-23 23:48:39,364 - INFO - Exiting...
